# 🚨 **Módulo 5: Detecção de Anomalias**

> **Tá, mas o que é Detecção de Anomalias?** É como ter um **detetive digital** que consegue identificar quando algo está "fora do normal"! Imagine que você tem um radar que detecta aviões inimigos - ele sabe como é o padrão normal de tráfego aéreo e dispara um alerta quando aparece algo suspeito.

---

## **Aula 5.1: O que é Detecção de Anomalias?**

### **Tá, mas como funciona isso?**

Detecção de Anomalias é como ter um **sistema imunológico digital** para sua IA. Ele aprende o que é "normal" e dispara alertas quando detecta algo que foge desse padrão.

**Analogia do dia a dia:**
Imagine que você é um porteiro de um prédio. Você conhece todos os moradores e seus horários. Se alguém que você nunca viu tentar entrar às 3h da manhã, você vai ficar desconfiado. Se o morador do apartamento 101, que sempre sai às 8h, aparecer às 2h da manhã, você também vai achar estranho!

**Por que isso é importante?**

Anomalias podem indicar:
- **Ataques de hackers** - Comportamento suspeito
- **Bugs no sistema** - Erros inesperados
- **Problemas de performance** - Lentidão anormal
- **Vazamento de dados** - Acesso não autorizado
- **Falhas de hardware** - Comportamento estranho

**Tipos de Anomalias:**
1. **Pontuais** - Um evento isolado estranho
2. **Contextuais** - Normal em um contexto, estranho em outro
3. **Coletivas** - Um grupo de eventos que juntos são suspeitos

---

**🖼️ Sugestão de imagem**: Um radar detectando pontos anômalos em um gráfico de dados normais

## **🔧 Setup Inicial - Preparando o Terreno**

Primeiro, vamos configurar nosso ambiente para detecção de anomalias:

In [ ]:
# 🔧 SETUP INICIAL - DETECÇÃO DE ANOMALIAS
import os
import re
import json
import logging
import random
import time
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, deque
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import warnings
warnings.filterwarnings('ignore')

# Configurando logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('deteccao_anomalias.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Configurando estilo dos gráficos
plt.style.use('default')
sns.set_palette("husl")

print("🚨 Ambiente de Detecção de Anomalias configurado!")
print("📝 Logs serão salvos em 'deteccao_anomalias.log'")

## ** Configurando o LLM**

Vamos configurar nosso LLM para testar técnicas de detecção de anomalias:

In [ ]:
# CONFIGURANDO O LLM
from dotenv import load_dotenv
load_dotenv()

def get_llm_colab():
    """Retorna o melhor LLM disponível no Colab"""
    
    # Tentativa 1: OpenAI
    try:
        from langchain_openai import ChatOpenAI
        api_key = os.getenv("OPENAI_API_KEY")
        if api_key:
            print(" Usando OpenAI GPT-3.5-turbo")
            return ChatOpenAI(
                model="gpt-3.5-turbo",
                temperature=0.7,
                api_key=api_key
            )
    except Exception as e:
        print(f"⚠️  OpenAI não disponível: {e}")
    
    # Tentativa 2: Hugging Face
    try:
        from langchain_community.llms import HuggingFaceHub
        token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
        if token:
            print("🤗 Usando Hugging Face (google/flan-t5-base)")
            return HuggingFaceHub(
                repo_id="google/flan-t5-base",
                model_kwargs={"temperature": 0.7, "max_length": 512}
            )
    except Exception as e:
        print(f"⚠️  Hugging Face não disponível: {e}")
    
    # Fallback: Simulação
    print("🔄 Usando modo simulação (sem API)")
    return None

# Configurando o LLM
llm = get_llm_colab()

if llm:
    print("✅ LLM configurado com sucesso!")
else:
    print("✅ Modo simulação ativado - você pode continuar o curso normalmente!")

## **�� Exemplo Prático 1: Detecção de Anomalias Simples**

Vamos começar com um exemplo simples: detectar usuários que estão fazendo muitas requisições em pouco tempo. É como identificar quando alguém está tentando "bombardear" o sistema:

In [ ]:
# 🚨 EXEMPLO 1: DETECÇÃO DE ANOMALIAS SIMPLES

class DetectorAnomaliasSimples:
    def __init__(self):
        # Histórico de requisições por usuário
        self.historico_usuarios = defaultdict(list)
        
        # Limites para detecção de anomalias
        self.limites = {
            'requisicoes_por_minuto': 10,  # Mais de 10 req/min é suspeito
            'requisicoes_por_hora': 100,   # Mais de 100 req/hora é suspeito
            'tempo_entre_requisicoes': 1   # Menos de 1 segundo entre req é suspeito
        }
        
        # Contador de anomalias detectadas
        self.anomalias_detectadas = 0
        self.usuarios_suspeitos = set()
    
    def registrar_requisicao(self, usuario, timestamp=None):
        """Registra uma nova requisição de usuário"""
        if timestamp is None:
            timestamp = time.time()
        
        self.historico_usuarios[usuario].append(timestamp)
        
        # Verificar anomalias
        anomalias = self.verificar_anomalias(usuario)
        
        if anomalias:
            self.anomalias_detectadas += 1
            self.usuarios_suspeitos.add(usuario)
            logger.warning(f"Anomalia detectada para usuário {usuario}: {anomalias}")
            
        return anomalias
    
    def verificar_anomalias(self, usuario):
        """Verifica se um usuário tem comportamento anômalo"""
        if usuario not in self.historico_usuarios:
            return []
        
        timestamps = self.historico_usuarios[usuario]
        agora = time.time()
        
        anomalias = []
        
        # Verificação 1: Requisições por minuto
        um_minuto_atras = agora - 60
        req_ultimo_minuto = len([t for t in timestamps if t > um_minuto_atras])
        
        if req_ultimo_minuto > self.limites['requisicoes_por_minuto']:
            anomalias.append(f"Muitas requisições por minuto: {req_ultimo_minuto}")
        
        # Verificação 2: Requisições por hora
        uma_hora_atras = agora - 3600
        req_ultima_hora = len([t for t in timestamps if t > uma_hora_atras])
        
        if req_ultima_hora > self.limites['requisicoes_por_hora']:
            anomalias.append(f"Muitas requisições por hora: {req_ultima_hora}")
        
        # Verificação 3: Tempo entre requisições
        if len(timestamps) >= 2:
            tempos_entre_req = [timestamps[i] - timestamps[i-1] for i in range(1, len(timestamps))]
            tempo_medio = np.mean(tempos_entre_req)
            
            if tempo_medio < self.limites['tempo_entre_requisicoes']:
                anomalias.append(f"Tempo muito curto entre requisições: {tempo_medio:.2f}s")
        
        return anomalias
    
    def get_estatisticas(self):
        """Retorna estatísticas de detecção"""
        return {
            'total_usuarios': len(self.historico_usuarios),
            'usuarios_suspeitos': len(self.usuarios_suspeitos),
            'anomalias_detectadas': self.anomalias_detectadas,
            'usuarios_suspeitos_lista': list(self.usuarios_suspeitos)
        }

# Testando o detector simples
print("🚨 Testando detector de anomalias simples...")
detector = DetectorAnomaliasSimples()

# Simulando comportamento normal
print("\n🟢 Simulando comportamento normal...")
for i in range(5):
    detector.registrar_requisicao("usuario_normal")
    time.sleep(2)  # 2 segundos entre requisições

# Simulando comportamento suspeito
print("\n🚨 Simulando comportamento suspeito...")
for i in range(15):  # Muitas requisições rapidamente
    detector.registrar_requisicao("usuario_suspeito")
    time.sleep(0.1)  # 0.1 segundos entre requisições

# Verificando estatísticas
print("\n📊 Estatísticas de detecção:")
estatisticas = detector.get_estatisticas()
for chave, valor in estatisticas.items():
    print(f"   {chave}: {valor}")

print("\n✅ Detector simples funcionando!")
print("💡 Anomalias detectadas baseadas em regras simples.")

### **🎯 Como funciona a detecção simples:**

O detector simples usa **regras baseadas em limites** para identificar anomalias. É como ter um radar que dispara um alerta quando algo passa de uma velocidade máxima.

**Vantagens:**
- Simples de implementar
- Fácil de entender
- Rápido de executar

**Desvantagens:**
- Não aprende padrões complexos
- Pode gerar falsos positivos
- Não se adapta a mudanças

---

**💡 Dica do Professor**: Detecção simples é como ter um alarme de carro - funciona bem para casos óbvios, mas pode não detectar ataques mais sofisticados!

## **�� Exemplo Prático 2: Detecção de Anomalias com Machine Learning**

Agora vamos usar Machine Learning para detectar anomalias. É como ter um detetive que aprende com a experiência e consegue identificar padrões sutis:

In [ ]:
# 🤖 EXEMPLO 2: DETECÇÃO DE ANOMALIAS COM MACHINE LEARNING

class DetectorAnomaliasML:
    def __init__(self):
        # Dados de treinamento (comportamento normal)
        self.dados_treinamento = []
        
        # Modelos de ML
        self.isolation_forest = IsolationForest(contamination=0.1, random_state=42)
        self.dbscan = DBSCAN(eps=0.5, min_samples=5)
        
        # Scaler para normalizar dados
        self.scaler = StandardScaler()
        
        # Modelo treinado
        self.modelo_treinado = False
        
        # Histórico de predições
        self.predicoes = []
        self.anomalias_detectadas = 0
    
    def gerar_dados_normais(self, n_amostras=1000):
        """Gera dados de comportamento normal"""
        print(f"📊 Gerando {n_amostras} amostras de comportamento normal...")
        
        dados = []
        for i in range(n_amostras):
            # Simulando características de requisições normais
            amostra = {
                'requisicoes_por_minuto': np.random.normal(5, 2),  # Média 5, desvio 2
                'tempo_entre_requisicoes': np.random.normal(12, 3),  # Média 12s, desvio 3
                'tamanho_requisicao': np.random.normal(500, 100),  # Média 500 bytes
                'hora_dia': np.random.uniform(0, 24),  # Hora do dia (0-24)
                'dia_semana': np.random.randint(0, 7),  # Dia da semana (0-6)
                'tipo_requisicao': np.random.choice([0, 1, 2, 3]),  # Tipos de requisição
                'erro_rate': np.random.beta(2, 98)  # Taxa de erro baixa
            }
            dados.append(amostra)
        
        self.dados_treinamento = dados
        return dados
    
    def preparar_features(self, dados):
        """Prepara features para o modelo"""
        features = []
        for amostra in dados:
            feature_vector = [
                amostra['requisicoes_por_minuto'],
                amostra['tempo_entre_requisicoes'],
                amostra['tamanho_requisicao'],
                amostra['hora_dia'],
                amostra['dia_semana'],
                amostra['tipo_requisicao'],
                amostra['erro_rate']
            ]
            features.append(feature_vector)
        
        return np.array(features)
    
    def treinar_modelo(self):
        """Treina o modelo de detecção de anomalias"""
        if not self.dados_treinamento:
            self.gerar_dados_normais()
        
        print("🤖 Treinando modelo de detecção de anomalias...")
        
        # Preparando dados
        X = self.preparar_features(self.dados_treinamento)
        
        # Normalizando dados
        X_scaled = self.scaler.fit_transform(X)
        
        # Treinando Isolation Forest
        self.isolation_forest.fit(X_scaled)
        
        # Treinando DBSCAN
        self.dbscan.fit(X_scaled)
        
        self.modelo_treinado = True
        print("✅ Modelo treinado com sucesso!")
        
        return X_scaled
    
    def detectar_anomalia(self, amostra):
        """Detecta se uma amostra é anômala"""
        if not self.modelo_treinado:
            raise Exception("Modelo não treinado!")
        
        # Preparando amostra
        features = self.preparar_features([amostra])
        features_scaled = self.scaler.transform(features)
        
        # Predições dos modelos
        pred_iso = self.isolation_forest.predict(features_scaled)[0]
        pred_dbscan = self.dbscan.fit_predict(features_scaled)[0]
        
        # Isolation Forest: -1 = anomalia, 1 = normal
        # DBSCAN: -1 = anomalia, outros valores = cluster normal
        
        is_anomalia_iso = pred_iso == -1
        is_anomalia_dbscan = pred_dbscan == -1
        
        # Score de confiança (média dos dois modelos)
        score_iso = self.isolation_forest.decision_function(features_scaled)[0]
        score_dbscan = 1 if pred_dbscan == -1 else 0
        
        confianca = (score_iso + score_dbscan) / 2
        
        resultado = {
            'is_anomalia': is_anomalia_iso or is_anomalia_dbscan,
            'confianca': confianca,
            'isolation_forest': is_anomalia_iso,
            'dbscan': is_anomalia_dbscan,
            'score_iso': score_iso,
            'score_dbscan': score_dbscan
        }
        
        self.predicoes.append(resultado)
        
        if resultado['is_anomalia']:
            self.anomalias_detectadas += 1
            logger.warning(f"Anomalia detectada com confiança {confianca:.2f}: {amostra}")
        
        return resultado
    
    def get_estatisticas(self):
        """Retorna estatísticas do modelo"""
        if not self.predicoes:
            return {"mensagem": "Nenhuma predição realizada ainda"}
        
        total_predicoes = len(self.predicoes)
        anomalias = sum(1 for p in self.predicoes if p['is_anomalia'])
        confianca_media = np.mean([p['confianca'] for p in self.predicoes])
        
        return {
            'total_predicoes': total_predicoes,
            'anomalias_detectadas': anomalias,
            'taxa_anomalias': anomalias / total_predicoes if total_predicoes > 0 else 0,
            'confianca_media': confianca_media,
            'modelo_treinado': self.modelo_treinado
        }

# Testando o detector com ML
print("🤖 Testando detector de anomalias com Machine Learning...")
detector_ml = DetectorAnomaliasML()

# Treinando o modelo
X_treino = detector_ml.treinar_modelo()

# Testando com dados normais
print("\n�� Testando com dados normais...")
dados_normais_teste = [
    {'requisicoes_por_minuto': 4, 'tempo_entre_requisicoes': 15, 'tamanho_requisicao': 450, 'hora_dia': 14, 'dia_semana': 2, 'tipo_requisicao': 1, 'erro_rate': 0.02},
    {'requisicoes_por_minuto': 6, 'tempo_entre_requisicoes': 10, 'tamanho_requisicao': 520, 'hora_dia': 16, 'dia_semana': 3, 'tipo_requisicao': 0, 'erro_rate': 0.01}
]

for i, amostra in enumerate(dados_normais_teste, 1):
    resultado = detector_ml.detectar_anomalia(amostra)
    print(f"{i}. Dados normais: {'🚨 ANOMALIA' if resultado['is_anomalia'] else '✅ NORMAL'} (Confiança: {resultado['confianca']:.2f})")

# Testando com dados anômalos
print("\n�� Testando com dados anômalos...")
dados_anomalos_teste = [
    {'requisicoes_por_minuto': 50, 'tempo_entre_requisicoes': 1, 'tamanho_requisicao': 2000, 'hora_dia': 3, 'dia_semana': 6, 'tipo_requisicao': 2, 'erro_rate': 0.8},
    {'requisicoes_por_minuto': 2, 'tempo_entre_requisicoes': 60, 'tamanho_requisicao': 50, 'hora_dia': 12, 'dia_semana': 1, 'tipo_requisicao': 3, 'erro_rate': 0.95}
]

for i, amostra in enumerate(dados_anomalos_teste, 1):
    resultado = detector_ml.detectar_anomalia(amostra)
    print(f"{i}. Dados anômalos: {'🚨 ANOMALIA' if resultado['is_anomalia'] else '✅ NORMAL'} (Confiança: {resultado['confianca']:.2f})")

# Verificando estatísticas
print("\n📊 Estatísticas do modelo ML:")
estatisticas_ml = detector_ml.get_estatisticas()
for chave, valor in estatisticas_ml.items():
    print(f"   {chave}: {valor}")

print("\n✅ Detector com ML funcionando!")
print("💡 Agora o sistema aprende padrões complexos automaticamente.")

## **�� Visualizando Detecção de Anomalias**

Vamos criar visualizações para entender melhor como funciona a detecção de anomalias:

In [ ]:
# �� VISUALIZANDO DETECÇÃO DE ANOMALIAS

# Gerando dados para visualização
np.random.seed(42)

# Dados normais
dados_normais = np.random.multivariate_normal(
    mean=[5, 12],  # Média de requisições/min e tempo entre req
    cov=[[2, 0.5], [0.5, 3]],  # Matriz de covariância
    size=200
)

# Dados anômalos
dados_anomalos = np.array([
    [25, 2],   # Muitas requisições, pouco tempo
    [1, 60],   # Poucas requisições, muito tempo
    [30, 0.5], # Muitas requisições, tempo muito baixo
    [0, 100],  # Nenhuma requisição, tempo muito alto
    [40, 1],   # Ataque DDoS
    [2, 80],   # Comportamento estranho
    [35, 0.1], # Ataque intenso
    [0.5, 120] # Comportamento muito lento
])

# Combinando dados
X_visualizacao = np.vstack([dados_normais, dados_anomalos])
labels = ['Normal'] * len(dados_normais) + ['Anômalo'] * len(dados_anomalos)

# Aplicando Isolation Forest
iso_forest = IsolationForest(contamination=0.1, random_state=42)
predicoes_iso = iso_forest.fit_predict(X_visualizacao)

# Aplicando DBSCAN
dbscan = DBSCAN(eps=2, min_samples=5)
predicoes_dbscan = dbscan.fit_predict(X_visualizacao)

# Criando gráficos
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Gráfico 1: Dados originais
cores_originais = ['green' if label == 'Normal' else 'red' for label in labels]
ax1.scatter(dados_normais[:, 0], dados_normais[:, 1], c='green', alpha=0.6, label='Normal', s=50)
ax1.scatter(dados_anomalos[:, 0], dados_anomalos[:, 1], c='red', alpha=0.8, label='Anômalo', s=100, marker='x')
ax1.set_title('Dados Originais (Normal vs Anômalo)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Requisições por Minuto')
ax1.set_ylabel('Tempo entre Requisições (s)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Gráfico 2: Isolation Forest
cores_iso = ['green' if pred == 1 else 'red' for pred in predicoes_iso]
ax2.scatter(X_visualizacao[:, 0], X_visualizacao[:, 1], c=cores_iso, alpha=0.6, s=50)
ax2.set_title('Isolation Forest - Detecção de Anomalias', fontsize=14, fontweight='bold')
ax2.set_xlabel('Requisições por Minuto')
ax2.set_ylabel('Tempo entre Requisições (s)')
ax2.grid(True, alpha=0.3)

# Adicionando legenda para Isolation Forest
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='green', alpha=0.6, label='Normal'),
                   Patch(facecolor='red', alpha=0.6, label='Anômalo')]
ax2.legend(handles=legend_elements)

# Gráfico 3: DBSCAN
cores_dbscan = ['blue' if pred == 0 else 'red' if pred == -1 else 'green' for pred in predicoes_dbscan]
ax3.scatter(X_visualizacao[:, 0], X_visualizacao[:, 1], c=cores_dbscan, alpha=0.6, s=50)
ax3.set_title('DBSCAN - Clustering e Anomalias', fontsize=14, fontweight='bold')
ax3.set_xlabel('Requisições por Minuto')
ax3.set_ylabel('Tempo entre Requisições (s)')
ax3.grid(True, alpha=0.3)

# Adicionando legenda para DBSCAN
legend_elements_dbscan = [Patch(facecolor='blue', alpha=0.6, label='Cluster Principal'),
                          Patch(facecolor='red', alpha=0.6, label='Anômalo'),
                          Patch(facecolor='green', alpha=0.6, label='Outros Clusters')]
ax3.legend(handles=legend_elements_dbscan)

# Gráfico 4: Comparação dos métodos
ax4.axis('off')
comparacao_texto = f"""
�� COMPARAÇÃO DOS MÉTODOS
{'='*40}

�� Isolation Forest:
   ✅ Detectou: {sum(1 for p in predicoes_iso if p == -1)} anomalias
   📈 Taxa de detecção: {sum(1 for p in predicoes_iso if p == -1)/len(predicoes_iso)*100:.1f}%
   💡 Baseado em isolamento de pontos

🔍 DBSCAN:
   ✅ Detectou: {sum(1 for p in predicoes_dbscan if p == -1)} anomalias
   📈 Taxa de detecção: {sum(1 for p in predicoes_dbscan if p == -1)/len(predicoes_dbscan)*100:.1f}%
   💡 Baseado em densidade de clusters

🎯 Dados Reais:
   🟢 Normais: {sum(1 for label in labels if label == 'Normal')}
   🚨 Anômalos: {sum(1 for label in labels if label == 'Anômalo')}

�� Conclusão:
   Ambos os métodos conseguem detectar
   a maioria das anomalias reais!
"""

ax4.text(0.1, 0.9, comparacao_texto, transform=ax4.transAxes, fontsize=12, 
         verticalalignment='top', fontfamily='monospace', fontweight='bold')

plt.tight_layout()
plt.show()

print("📊 Visualização de detecção de anomalias concluída!")
print("💡 Os gráficos mostram como diferentes algoritmos detectam anomalias.")

## **🔍 Teste Rápido - Identificando Anomalias**

Agora é sua vez! Vamos testar se você consegue identificar anomalias em diferentes cenários:

In [ ]:
# 🔍 TESTE RÁPIDO - IDENTIFICANDO ANOMALIAS

def testar_identificacao_anomalias():
    """Testa a capacidade de identificar anomalias"""
    
    # Cenários de comportamento
    cenarios = [
        {
            'nome': 'Comportamento Normal',
            'requisicoes_por_minuto': 5,
            'tempo_entre_requisicoes': 12,
            'tamanho_requisicao': 500,
            'hora_dia': 14,
            'erro_rate': 0.02,
            'descricao': 'Usuário normal durante o dia'
        },
        {
            'nome': 'Ataque DDoS',
            'requisicoes_por_minuto': 100,
            'tempo_entre_requisicoes': 0.5,
            'tamanho_requisicao': 1000,
            'hora_dia': 3,
            'erro_rate': 0.1,
            'descricao': 'Muitas requisições rapidamente'
        },
        {
            'nome': 'Bug no Sistema',
            'requisicoes_por_minuto': 3,
            'tempo_entre_requisicoes': 20,
            'tamanho_requisicao': 400,
            'hora_dia': 10,
            'erro_rate': 0.8,
            'descricao': 'Muitos erros no sistema'
        },
        {
            'nome': 'Acesso Não Autorizado',
            'requisicoes_por_minuto': 8,
            'tempo_entre_requisicoes': 8,
            'tamanho_requisicao': 2000,
            'hora_dia': 2,
            'erro_rate': 0.6,
            'descricao': 'Tentativas de acesso suspeito'
        },
        {
            'nome': 'Usuário Inativo',
            'requisicoes_por_minuto': 0.5,
            'tempo_entre_requisicoes': 120,
            'tamanho_requisicao': 200,
            'hora_dia': 18,
            'erro_rate': 0.01,
            'descricao': 'Usuário muito inativo'
        }
    ]
    
    classificacoes_corretas = [
        "Normal", "Anômalo", "Anômalo", "Anômalo", "Anômalo"
    ]
    
    print("🔍 TESTE DE IDENTIFICAÇÃO DE ANOMALIAS")
    print("=" * 60)
    print("\nAnalise cada cenário e classifique como:")
    print("   🟢 Normal - Comportamento esperado")
    print("   🚨 Anômalo - Comportamento suspeito ou estranho")
    
    acertos = 0
    total = len(cenarios)
    
    for i, (cenario, classificacao_correta) in enumerate(zip(cenarios, classificacoes_corretas), 1):
     print(f"\n{i}. Cenário: {cenario['nome']}")
        print(f"   �� Requisições/min: {cenario['requisicoes_por_minuto']}")
        print(f"   ⏱️ Tempo entre req: {cenario['tempo_entre_requisicoes']}s")
        print(f"   📦 Tamanho req: {cenario['tamanho_requisicao']} bytes")
        print(f"   🕐 Hora: {cenario['hora_dia']}h")
        print(f"   ❌ Taxa erro: {cenario['erro_rate']*100:.1f}%")
        print(f"   📝 Descrição: {cenario['descricao']}")
        
        # Simulando análise automática
        requisicoes = cenario['requisicoes_por_minuto']
        tempo = cenario['tempo_entre_requisicoes']
        erro = cenario['erro_rate']
        hora = cenario['hora_dia']
        
        # Regras de detecção
        if requisicoes > 50 or tempo < 1 or erro > 0.5:
            analise_auto = "Anômalo"
        elif requisicoes < 1 or tempo > 60:
            analise_auto = "Anômalo"
        elif (hora < 6 or hora > 22) and requisicoes > 10:
            analise_auto = "Anômalo"
        else:
            analise_auto = "Normal"
        
        print(f"   Análise Automática: {analise_auto}")
        
        # Verificando acerto
        if analise_auto == classificacao_correta:
            acertos += 1
            print(f"   ✅ CORRETO!")
        else:
            print(f"   ❌ INCORRETO! Classificação correta: {classificacao_correta}")
    
    # Resultado final
    print(f"\n RESULTADO FINAL:")
    print(f"   Acertos: {acertos}/{total}")
    print(f"   Taxa de acerto: {(acertos/total)*100:.1f}%")
    
    if acertos >= total * 0.8:
        print("   🏆 EXCELENTE! Você tem um bom olho para detectar anomalias!")
    elif acertos >= total * 0.6:
        print("   👍 BOM! Você está no caminho certo!")
    else:
        print("   Continue estudando! A prática leva à perfeição!")
    
    return acertos, total

# Executando o teste
acertos, total = testar_identificacao_anomalias()

# Logging do resultado
logger.info(f"Teste de identificação de anomalias concluído: {acertos}/{total} acertos")

## **🏆 Desafio do Módulo - Sistema de Detecção de Anomalias Avançado**

Agora é hora de colocar em prática! Vamos criar um sistema completo de detecção de anomalias:

In [ ]:
# 🏆 DESAFIO DO MÓDULO - SISTEMA DE DETECÇÃO DE ANOMALIAS AVANÇADO

class SistemaDetecaoAnomaliasAvancado:
    def __init__(self):
        # TODO: Implemente as variáveis necessárias
        # - Múltiplos algoritmos de detecção
        # - Sistema de scoring de confiança
        # - Aprendizado contínuo
        # - Alertas inteligentes
        # - Dashboard de anomalias
        pass
    
    def configurar_algoritmos(self):
        """Configura múltiplos algoritmos de detecção"""
        # TODO: Implemente algoritmos
        # - Isolation Forest
        # - DBSCAN
        # - One-Class SVM
        # - Autoencoder
        # - LSTM para séries temporais
        pass
    
    def sistema_voting(self, predicoes):
        """Sistema de votação entre algoritmos"""
        # TODO: Implemente votação
        # - Peso diferente para cada algoritmo
        # - Consenso entre algoritmos
        # - Score de confiança
        # - Redução de falsos positivos
        pass
    
    def aprendizado_continuo(self, dados_novos):
        """Aprendizado contínuo com novos dados"""
        # TODO: Implemente aprendizado
        # - Atualização incremental dos modelos
        # - Adaptação a mudanças de padrão
        # - Detecção de concept drift
        # - Retreinamento automático
        pass
    
    def alertas_inteligentes(self, anomalia):
        """Sistema de alertas inteligentes"""
        # TODO: Implemente alertas
        # - Diferentes níveis de severidade
        # - Agrupamento de alertas similares
        # - Escalação automática
        # - Notificações personalizadas
        pass
    
    def dashboard_anomalias(self):
        """Dashboard de anomalias em tempo real"""
        # TODO: Implemente dashboard
        # - Visualização de anomalias
        # - Métricas de performance
        # - Histórico de detecções
        # - Análise de tendências
        pass

print(" DESAFIO: Implemente o sistema de detecção de anomalias avançado acima!")
print("💡 Dicas:")
print("   - Use ensemble de múltiplos algoritmos")
print("   - Implemente sistema de votação ponderada")
print("   - Configure aprendizado contínuo")
print("   - Crie alertas inteligentes e escaláveis")
print("   - Desenvolva dashboard em tempo real")
print("   - Implemente detecção de concept drift")
print("   - Use deep learning para padrões complexos")

## **�� Dashboard de Detecção de Anomalias**

Vamos criar um dashboard completo para monitoramento de anomalias:

In [ ]:
# �� DASHBOARD DE DETECÇÃO DE ANOMALIAS

# Dados simulados de anomalias
dados_metricas = {
    'total_requisicoes': 15420,
    'anomalias_detectadas': 127,
    'taxa_deteccao': 98.2,
    'falsos_positivos': 8,
    'falsos_negativos': 3,
    'precisao': 93.7,
    'recall': 97.7,
    'algoritmos_ativos': 4
}

# Tipos de anomalias detectadas
tipos_anomalias = {
    'Ataque DDoS': 45,
    'Acesso Não Autorizado': 32,
    'Bug no Sistema': 28,
    'Performance Degradada': 15,
    'Comportamento Estranho': 7
}

# Performance dos algoritmos
algoritmos = ['Isolation Forest', 'DBSCAN', 'One-Class SVM', 'Autoencoder']
precisao_algoritmos = [94.2, 91.8, 96.1, 92.5]
recall_algoritmos = [97.3, 95.1, 98.2, 93.8]

# Criando dashboard
fig = plt.figure(figsize=(16, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Título principal
fig.suptitle('DASHBOARD DE DETECÇÃO DE ANOMALIAS - SISTEMA DE IA', fontsize=20, fontweight='bold')

# Métricas principais (2x2 grid)
ax1 = fig.add_subplot(gs[0, 0])
ax1.text(0.5, 0.8, f"{dados_metricas['anomalias_detectadas']}", ha='center', va='center', fontsize=24, fontweight='bold', color='#ff6b6b')
ax1.text(0.5, 0.5, 'ANOMALIAS DETECTADAS', ha='center', va='center', fontsize=12)
ax1.text(0.5, 0.2, f"Taxa: {dados_metricas['taxa_deteccao']}%", ha='center', va='center', fontsize=10, color='gray')
ax1.set_xlim(0, 1)
ax1.set_ylim(0, 1)
ax1.axis('off')

ax2 = fig.add_subplot(gs[0, 1])
ax2.text(0.5, 0.8, f"{dados_metricas['precisao']}%", ha='center', va='center', fontsize=24, fontweight='bold', color='#4ecdc4')
ax2.text(0.5, 0.5, 'PRECISÃO', ha='center', va='center', fontsize=12)
ax2.text(0.5, 0.2, f"Falsos +: {dados_metricas['falsos_positivos']}", ha='center', va='center', fontsize=10, color='gray')
ax2.set_xlim(0, 1)
ax2.set_ylim(0, 1)
ax2.axis('off')

ax3 = fig.add_subplot(gs[0, 2])
ax3.text(0.5, 0.8, f"{dados_metricas['recall']}%", ha='center', va='center', fontsize=24, fontweight='bold', color='#feca57')
ax3.text(0.5, 0.5, 'RECALL', ha='center', va='center', fontsize=12)
ax3.text(0.5, 0.2, f"Falsos -: {dados_metricas['falsos_negativos']}", ha='center', va='center', fontsize=10, color='gray')
ax3.set_xlim(0, 1)
ax3.set_ylim(0, 1)
ax3.axis('off')

ax4 = fig.add_subplot(gs[0, 3])
ax4.text(0.5, 0.8, f"{dados_metricas['algoritmos_ativos']}", ha='center', va='center', fontsize=24, fontweight='bold', color='#96ceb4')
ax4.text(0.5, 0.5, 'ALGORITMOS ATIVOS', ha='center', va='center', fontsize=12)
ax4.text(0.5, 0.2, 'Ensemble', ha='center', va='center', fontsize=10, color='gray')
ax4.set_xlim(0, 1)
ax4.set_ylim(0, 1)
ax4.axis('off')

# Tipos de anomalias
ax5 = fig.add_subplot(gs[1, :2])
tipos = list(tipos_anomalias.keys())
contagens = list(tipos_anomalias.values())
cores = ['#ff6b6b', '#feca57', '#45b7d1', '#96ceb4', '#4ecdc4']

bars = ax5.bar(tipos, contagens, color=cores, alpha=0.8)
ax5.set_title('Tipos de Anomalias Detectadas', fontsize=14, fontweight='bold')
ax5.set_ylabel('Número de Detecções')
ax5.tick_params(axis='x', rotation=45)

for bar, valor in zip(bars, contagens):
    ax5.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             str(valor), ha='center', va='bottom', fontweight='bold')

# Performance dos algoritmos
ax6 = fig.add_subplot(gs[1, 2:])
x = np.arange(len(algoritmos))
width = 0.35

bars1 = ax6.bar(x - width/2, precisao_algoritmos, width, label='Precisão', color='#4ecdc4', alpha=0.8)
bars2 = ax6.bar(x + width/2, recall_algoritmos, width, label='Recall', color='#feca57', alpha=0.8)

ax6.set_title('Performance dos Algoritmos (%)', fontsize=14, fontweight='bold')
ax6.set_ylabel('Performance (%)')
ax6.set_xticks(x)
ax6.set_xticklabels(algoritmos, rotation=45)
ax6.legend()
ax6.set_ylim(85, 100)

# Adicionando valores nas barras
for bar in bars1:
    height = bar.get_height()
    ax6.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'{height:.1f}%', ha='center', va='bottom', fontsize=9)

for bar in bars2:
    height = bar.get_height()
    ax6.text(bar.get_x() + bar.get_width()/2., height + 0.5,
             f'{height:.1f}%', ha='center', va='bottom', fontsize=9)

# Resumo do sistema
ax7 = fig.add_subplot(gs[2, :])
ax7.axis('off')

resumo_texto = f"""
📊 RESUMO DO SISTEMA DE DETECÇÃO
{'='*50}

🎯 Total de Requisições Analisadas: {dados_metricas['total_requisicoes']:,}
🚨 Anomalias Detectadas: {dados_metricas['anomalias_detectadas']}
📈 Taxa de Detecção: {dados_metricas['taxa_deteccao']}%
✅ Precisão: {dados_metricas['precisao']}% (Falsos Positivos: {dados_metricas['falsos_positivos']})
🔄 Recall: {dados_metricas['recall']}% (Falsos Negativos: {dados_metricas['falsos_negativos']})

🤖 Algoritmos em Ensemble: {dados_metricas['algoritmos_ativos']}
⚡ Status: SISTEMA OPERACIONAL
🛡️ Última Atualização: {datetime.now().strftime('%d/%m/%Y %H:%M')}

💡 O sistema está detectando anomalias com alta precisão e recall!
"""

ax7.text(0.05, 0.95, resumo_texto, transform=ax7.transAxes, fontsize=12, 
         verticalalignment='top', fontfamily='monospace', fontweight='bold')

plt.show()

print("�� Dashboard de Detecção de Anomalias gerado com sucesso!")
print(f"🎯 {dados_metricas['anomalias_detectadas']} anomalias detectadas")
print(f" Sistema de detecção funcionando com alta precisão!")

---

## ** Resumo do Módulo**

Neste módulo, aprendemos:

### **�� Conceitos de Detecção de Anomalias:**
- **O que é Detecção de Anomalias** - Identificação de comportamentos anômalos
- **Tipos de anomalias** - Pontuais, contextuais e coletivas
- **Algoritmos de ML** - Isolation Forest, DBSCAN, One-Class SVM
- **Ensemble de modelos** - Combinação de múltiplos algoritmos

### **🛠️ Ferramentas Aprendidas:**
- Detecção baseada em regras simples
- Machine Learning para detecção avançada
- Sistema de votação entre algoritmos
- Dashboard de monitoramento
- Métricas de performance (precisão, recall)

### **🔬 Técnicas Práticas:**
- Como implementar detecção simples
- Como usar ML para detecção avançada
- Como visualizar anomalias
- Como avaliar performance dos algoritmos
- Como criar dashboards de monitoramento

### ** Próximos Passos:**
- No próximo módulo, vamos fazer o **Projeto Final**
- Integração de todas as técnicas aprendidas
- Sistema completo de segurança e observabilidade
- Deploy em ambiente real

---

**💡 Dica do Professor**: Detecção de Anomalias é como ter um "sexto sentido digital" - você consegue identificar problemas antes que eles causem danos!

**🚀 Próximo módulo**: [Módulo 6: Projeto Final - Sistema Completo](./06_projeto_final.ipynb)

---

**🎉 Parabéns! Você completou o Módulo 5!** 🚨

**🏆 Conquista Desbloqueada**: "Detetive Digital" - Você agora pode detectar anomalias em sistemas de IA com alta precisão!